Glove + LSTM

In [ ]:
!pip install spacy tensorflow
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 1.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from concurrent.futures import thread
from flask import Flask, request, jsonify
#from flask_ngrok import run_with_ngrok
from tensorflow.keras.models import load_model
import spacy
import threading
import tensorflow as tf

# Load Glove (via Spacy)
nlp=spacy.load("en_core_web_lg")
MAX_LEN,EMBED_DIM = 50,300

def text_to_glove_tensor(text: str) ->np.ndarray:
  doc=nlp(text)
  vecs = [tok.vector for tok in doc]
  tensor = np.zeros((MAX_LEN,EMBED_DIM),dtype=np.float32)
  length = min(len(vecs),MAX_LEN)
  if length:
    tensor[:length]=vecs[:length]
  return np.expand_dims(tensor,axis=0) # shape: (1,MAX_LEN, 300) (1,4,300)

#Load model
model = load_model("/content/drive/MyDrive/Itvedant/NLP/glove_lstm_sentiment_analysis.h5") # upload or mount model

# Flask setup
app = Flask(__name__)
#run_with_ngrok(app)

@app.route("/")
def index():
  return "Glove - Lstm Sentiment APU running!"

@app.route("/predict", methods=["POST"])
def predict():
  data = request.get_json(force=True)
  if not data or "text" not in data:
    return jsonify({"error":"Missing 'text' field"}),400

  text = data["text"]
  tensor = text_to_glove_tensor(text)
  prob = float(model.predict(tensor, verbose=0)[0][0])
  pred = int(prob >= 0.5)

  return jsonify({"probability": prob, "prediction":pred})

def run_flask():
  app.run(port=5050)


# Start Flask app in background
thread = threading.Thread(target=run_flask)
thread.start()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5050
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
import requests
response = requests.post("http://127.0.0.1:5050/predict", json={"text": "This is not a very bad movie waste of time "})
print(response.json())

INFO:werkzeug:127.0.0.1 - - [12/Jul/2025 06:07:15] "POST /predict HTTP/1.1" 200 -


{'prediction': 0, 'probability': 0.03666869178414345}
